# GD en el perceptrón

El método de gradiente descendiente es un método de optimización que se basa en observar el gradiente de una función (idealmente convexa) para encontrar el mínimo de esta función. Aquí veremos cómo se puede aplicar este método al perceptrón.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Preparación de los datos

In [2]:
data = pd.read_csv('cat_data.csv')
data

,¿es animal?,¿es mamífero?,¿es felino?,¿es doméstico?,¿tiene dos orejas?,¿es negro?,¿tiene cuatro patas?,¿es gato?
0,1,1,1,1,1,1,1,1
1,0,0,0,1,0,1,0,0
2,1,0,1,1,0,1,1,0
3,1,1,0,1,1,0,1,0
4,1,1,1,0,1,0,1,0
5,1,1,1,1,0,0,0,1
6,1,0,0,1,1,1,0,0
7,1,1,1,1,0,0,1,1
8,1,0,0,1,0,0,0,0
9,0,0,0,0,0,0,0,0


In [3]:
#Convertir los datos a numpy
npData = data.to_numpy()
#Ejemplos
X = npData[:,:-1]
#Clases de los ejemplos
Y = npData[:,-1]

#Tamaño de los datos
#Unidades de entrada
m,n = X.shape

## Aprendizaje por GD

Determinación de hiperparámetros:

In [4]:
#Número de itraciones
its = 100
#Rango de aprendizaje
lr = 0.2

#### Entrenamiento de la red

Determinaremos los pesos a partir del algoritmo de gradiente descendiente:

$$\theta_i \leftarrow \theta_i - \eta \nabla_iR(\theta)$$

En este caso, la función de riesgo es:

$$R(\theta) = -\sum_x \sum_y y \log p(Y=y|x)$$

Donde $y$ es la clase de los datos. Además, la probabilidad se determinará por la función Softmax y $\theta = \{w,b: w \in \mathbb{R}^d, b \in \mathbb{R}\}$. Por tanto:

$$\nabla_iR(\theta) = (p(Y=y|x) - y_x) \cdot x_i$$

In [5]:
%%time

np.random.seed(0)
w = np.random.rand(n,2)/np.sqrt(n)
b = np.ones(2)

#Detenerse
stop = False
t = 0
while  stop == False:
    #FORWARD
    #Funcion de preactivacióm
    a = np.dot(X,w)+b
    #Función de activación
    exp = np.exp(a-np.max(a))
    pred = exp/exp.sum(1,keepdims=True)

    #Error
    pred[range(m),Y] -= Y
    #Derivada
    DW = np.dot(X.T,pred)
    Db = pred.sum(0)
    
    #ACTUALIZACIÓN
    #Gradiente descendiente
    w -= lr*DW
    b -= lr*Db
    
    t += 1
    #Criterio de paro
    if t > its: 
        stop = True

CPU times: user 10.2 ms, sys: 0 ns, total: 10.2 ms
Wall time: 27.8 ms


Aplicación de la red a los datos: La elección de una clase se hace como:

$$\hat{y} = \arg\max_y p(Y=y|x)$$

In [6]:
#Predicción
def forward(X):
    #Pre-activación
    a = np.dot(X,w)+b
    #Activación
    exp = np.exp(a)
    f = exp/exp.sum(1,keepdims=True)
    #Clasificación
    cl = np.argmax(f, axis=1)
    
    return cl, f

In [7]:
clases, probs = forward(X)
print('Clases predichas: {} \nClases esperadas: {} \nError: {}'.format(clases,Y, np.abs(clases-Y).sum(0)/len(Y)))

Clases predichas: [0 0 0 0 0 1 0 1 0 0 0 0 1 1] 
Clases esperadas: [1 0 0 0 0 1 0 1 0 0 0 0 1 1] 
Error: 0.07142857142857142


#### Exploración de los pesos

El perceptrón que hemos definido, determina las probabilidades para la clase 1 (gato) y la clase 0 (no gato) y elige la clase que maximice la probabilidad.

Las probabilidades de salida son:

In [18]:
pd.DataFrame(data=probs, columns=['Prob. clase 0', 'Prob. clase 1'])

,Prob. clase 0,Prob. clase 1
0,0.563958,0.436042
1,0.571438,0.428562
2,0.937708,0.062292
3,0.924006,0.075994
4,0.868989,0.131011
5,0.006439,0.993561
6,0.970421,0.029579
7,0.159770,0.840230
8,0.828278,0.171722
9,0.872420,0.127580


Los pesos de las conexiones que ha aprendido para la clase 1 (gato) son los siguientes:

In [34]:
pd.DataFrame(data=np.append(w[:,1],b[1]), index=list(data.columns)[:7]+['bias'], columns=['Pesos de clase gato'])

,Pesos de clase gato
¿es animal?,-71.895930
¿es mamífero?,-38.789271
¿es felino?,-38.285538
¿es doméstico?,-48.639444
¿tiene dos orejas?,-50.412084
¿es negro?,-40.942354
¿tiene cuatro patas?,-51.907203
bias,-90.861263
